# Activity Finder Agent - Google ADK Implementation

This notebook implements an Activity Finder agent using Google ADK (Agent Development Kit) with dual search capabilities:
- **Google Search**: Primary search tool for activity research
- **Tavily Search**: Additional search for enrichment and verification

## Input Format
```python
{
  "activities": ["Activity 1", "Activity 2", ...],  # List of activity names
  "city": "City Name",
  "num_days": 3,
  "budget_per_person": 300.0
}
```

## Output Format
```python
[
  {
    "title": "Activity Name",
    "price": 35.00,
    "description": "Detailed description...",
    "duration": "2-3 hours",
    "category": "sightseeing"
  },
  ...
]
```

## Section 1: Setup & Imports

In [ ]:
# Standard library imports
import sys
import os
import json
import uuid
import re
from typing import Dict, Any, List

# Add backend to path for imports
sys.path.append('./backend')

# Google ADK imports
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, FunctionTool
from google.genai import types

# Import from backend
from tools.activity_search import tavily_search_activities
from agents.utils.data_models import ActivityDetail, ActivitySearchResults

# For nice output display
import pandas as pd
from IPython.display import display, HTML

print("✓ All imports successful!")

## Section 2: Environment Setup & API Key Validation

In [ ]:
# Check for required API keys
def check_api_keys():
    """Validate that required API keys are set in environment."""
    keys_status = {
        "GOOGLE_API_KEY": os.environ.get("GOOGLE_API_KEY"),
        "TAVILY_API_KEY": os.environ.get("TAVILY_API_KEY")
    }
    
    all_present = True
    for key_name, key_value in keys_status.items():
        if key_value:
            print(f"✓ {key_name}: Set")
        else:
            print(f"✗ {key_name}: NOT SET - Required for agent to function")
            all_present = False
    
    if not all_present:
        print("\n⚠️  Please set missing API keys:")
        print("   export GOOGLE_API_KEY='your-key-here'")
        print("   export TAVILY_API_KEY='your-key-here'")
        return False
    
    print("\n✓ All API keys configured!")
    return True

# Run check
api_keys_valid = check_api_keys()

# Model configuration
MODEL_NAME = "gemini-2.5-flash"
APP_NAME = "activity_finder_notebook"
USER_ID = "notebook_user"

print(f"\nModel: {MODEL_NAME}")
print(f"App: {APP_NAME}")

## Section 3: Load Agent Instructions

In [ ]:
# Load instructions from file
instructions_path = "backend/agents/phase2/activity_finder/activity_finder_instructions.txt"

with open(instructions_path, 'r') as f:
    activity_finder_instructions = f.read()

print(f"✓ Loaded instructions from {instructions_path}")
print(f"  Instructions length: {len(activity_finder_instructions)} characters")
print("\n--- First 500 characters ---")
print(activity_finder_instructions[:500])
print("...")

## Section 4: Tool Integration - Google Search & Tavily Search

In [ ]:
# Google Search tool is already available from google.adk.tools
print("✓ Google Search tool: Available via google_search")

# Create FunctionTool wrapper for Tavily search
# FunctionTool automatically derives name and description from the function
tavily_tool = FunctionTool(tavily_search_activities)

print("✓ Tavily Search tool: Wrapped as FunctionTool")
print("\nAvailable tools for agent:")
print("  1. google_search - Primary search for activity research")
print("  2. tavily_search_activities - Secondary search for enrichment")

## Section 5: Create Activity Finder Agent

In [ ]:
# Create the Activity Finder Agent with both search tools
activity_finder_agent = Agent(
    name="activity_finder_agent",
    model=MODEL_NAME,
    instruction=activity_finder_instructions,
    description="Research activities with pricing, duration, and detailed descriptions for travel planning.",
    tools=[google_search, tavily_tool]  # Both search tools available
)

print("✓ Activity Finder Agent created successfully!")
print(f"  Name: {activity_finder_agent.name}")
print(f"  Model: {MODEL_NAME}")
print(f"  Tools: {len(activity_finder_agent.tools)} tools attached")

## Section 6: Runner & Session Management

In [ ]:
# Create session service and runner
session_service = InMemorySessionService()

activity_finder_runner = Runner(
    agent=activity_finder_agent,
    app_name=APP_NAME,
    session_service=session_service,
)

print("✓ Session service initialized")
print("✓ Runner created and bound to activity_finder_agent")
print("\nReady to process activity search requests!")

## Section 7: Main Execution Function

In [ ]:
async def find_activities(input_data: Dict[str, Any], debug: bool = False) -> Dict[str, Any]:
    """
    Execute the Activity Finder agent to research activities.
    
    Args:
        input_data: Dict with keys: activities (list), city (str), num_days (int), budget_per_person (float)
        debug: If True, print debugging information
    
    Returns:
        Dict with destination, activities list, and metadata
    """
    try:
        # Create new session
        session_id = str(uuid.uuid4())
        
        await session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=session_id,
        )
        
        if debug:
            print(f"Session ID: {session_id}")
            print(f"Input data: {json.dumps(input_data, indent=2)}")
        
        # Convert input to JSON string for agent
        user_text = json.dumps(input_data)
        user_content = types.Content(
            role="user",
            parts=[types.Part(text=user_text)],
        )
        
        # Run agent
        events = activity_finder_runner.run_async(
            user_id=USER_ID,
            session_id=session_id,
            new_message=user_content,
        )
        
        # Collect response
        final_text = ""
        async for event in events:
            if hasattr(event, 'content') and event.content:
                for part in event.content.parts:
                    if hasattr(part, 'text'):
                        final_text += part.text
                        if debug:
                            print(f"[Agent chunk]: {part.text[:100]}...")
        
        if debug:
            print(f"\n=== Full Agent Response ===")
            print(final_text)
            print(f"\n=== End Response ===")
        
        # Extract JSON from response
        json_match = re.search(r'\{.*\}', final_text, re.DOTALL)
        if not json_match:
            raise ValueError(f"No JSON found in agent response: {final_text}")
        
        json_str = json_match.group(0)
        result_dict = json.loads(json_str)
        
        # Validate with Pydantic model
        activity_results = ActivitySearchResults(**result_dict)
        
        # Return structured result
        return {
            "status": "success",
            "destination": activity_results.destination,
            "activities": [
                {
                    "name": activity.name,
                    "description": activity.description,
                    "estimated_duration": activity.estimated_duration,
                    "estimated_cost_per_person": activity.estimated_cost_per_person,
                    "category": activity.category,
                }
                for activity in activity_results.activities
            ],
            "raw_response": final_text if debug else None,
        }
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        
        return {
            "status": "error",
            "destination": input_data.get("city", "Unknown"),
            "activities": [],
            "error": str(e),
        }

print("✓ find_activities() function defined")

## Section 8: Output Formatting Function

In [ ]:
def format_output(result: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Transform agent result into requested format with title, price, description, duration.
    
    Args:
        result: Result dict from find_activities()
    
    Returns:
        List of dicts with keys: title, price, description, duration, category
    """
    if result["status"] != "success":
        print(f"❌ Error in result: {result.get('error', 'Unknown error')}")
        return []
    
    formatted = []
    for activity in result["activities"]:
        formatted.append({
            "title": activity["name"],
            "price": activity["estimated_cost_per_person"],
            "description": activity["description"],
            "duration": activity["estimated_duration"],
            "category": activity["category"],
        })
    
    return formatted

def calculate_budget_summary(formatted_activities: List[Dict], budget: float) -> Dict:
    """
    Calculate budget summary from formatted activities.
    
    Returns:
        Dict with total_cost, budget, remaining, over_budget flag
    """
    total_cost = sum(a["price"] for a in formatted_activities)
    remaining = budget - total_cost
    
    return {
        "total_cost": round(total_cost, 2),
        "budget": budget,
        "remaining": round(remaining, 2),
        "over_budget": remaining < 0,
    }

def display_results(formatted_activities: List[Dict], budget_summary: Dict, destination: str):
    """
    Display results in a nice formatted table.
    """
    print(f"\n{'='*80}")
    print(f"🌍 ACTIVITY SEARCH RESULTS: {destination}")
    print(f"{'='*80}\n")
    
    # Create DataFrame for nice display
    df = pd.DataFrame(formatted_activities)
    
    # Reorder columns
    df = df[['title', 'category', 'price', 'duration', 'description']]
    
    # Format price
    df['price'] = df['price'].apply(lambda x: f"${x:.2f}")
    
    display(df)
    
    # Budget summary
    print(f"\n{'='*80}")
    print("💰 BUDGET SUMMARY")
    print(f"{'='*80}")
    print(f"Total Cost:       ${budget_summary['total_cost']:.2f}")
    print(f"Budget:           ${budget_summary['budget']:.2f}")
    print(f"Remaining:        ${budget_summary['remaining']:.2f}")
    
    if budget_summary['over_budget']:
        print(f"\n⚠️  WARNING: Over budget by ${abs(budget_summary['remaining']):.2f}!")
    else:
        print(f"\n✓ Within budget with ${budget_summary['remaining']:.2f} to spare")
    
    print(f"{'='*80}\n")

print("✓ Output formatting functions defined")

## Section 9: Example Usage

### Example 1: Paris Activities - 3 Days, $300 Budget

In [ ]:
# Example 1: Paris
example1_input = {
    "activities": [
        "Eiffel Tower visit",
        "Louvre Museum",
        "Seine River cruise",
        "Arc de Triomphe",
        "Montmartre walking tour"
    ],
    "city": "Paris",
    "num_days": 3,
    "budget_per_person": 300.0
}

print("🔍 Searching for activities in Paris...")
print(f"Activities: {', '.join(example1_input['activities'])}")
print(f"Duration: {example1_input['num_days']} days")
print(f"Budget: ${example1_input['budget_per_person']} per person\n")

# Run the agent
result1 = await find_activities(example1_input, debug=False)

# Format and display
if result1["status"] == "success":
    formatted1 = format_output(result1)
    budget_summary1 = calculate_budget_summary(formatted1, example1_input["budget_per_person"])
    display_results(formatted1, budget_summary1, result1["destination"])
else:
    print(f"❌ Error: {result1.get('error', 'Unknown error')}")

### Example 2: Tokyo Activities - 5 Days, $500 Budget

In [ ]:
# Example 2: Tokyo
example2_input = {
    "activities": [
        "Tokyo Skytree observation deck",
        "Senso-ji Temple",
        "Tsukiji Fish Market food tour",
        "Shibuya Crossing and shopping",
        "TeamLab Borderless digital art museum",
        "Meiji Shrine",
        "Day trip to Mount Fuji"
    ],
    "city": "Tokyo",
    "num_days": 5,
    "budget_per_person": 500.0
}

print("🔍 Searching for activities in Tokyo...")
print(f"Activities: {', '.join(example2_input['activities'])}")
print(f"Duration: {example2_input['num_days']} days")
print(f"Budget: ${example2_input['budget_per_person']} per person\n")

# Run the agent
result2 = await find_activities(example2_input, debug=False)

# Format and display
if result2["status"] == "success":
    formatted2 = format_output(result2)
    budget_summary2 = calculate_budget_summary(formatted2, example2_input["budget_per_person"])
    display_results(formatted2, budget_summary2, result2["destination"])
else:
    print(f"❌ Error: {result2.get('error', 'Unknown error')}")

### Example 3: Barcelona Activities - 4 Days, $400 Budget

In [ ]:
# Example 3: Barcelona
example3_input = {
    "activities": [
        "Sagrada Familia tour",
        "Park Güell",
        "Gothic Quarter walking tour",
        "La Rambla and Boqueria Market",
        "Casa Batlló",
        "Beach day at Barceloneta"
    ],
    "city": "Barcelona",
    "num_days": 4,
    "budget_per_person": 400.0
}

print("🔍 Searching for activities in Barcelona...")
print(f"Activities: {', '.join(example3_input['activities'])}")
print(f"Duration: {example3_input['num_days']} days")
print(f"Budget: ${example3_input['budget_per_person']} per person\n")

# Run the agent
result3 = await find_activities(example3_input, debug=False)

# Format and display
if result3["status"] == "success":
    formatted3 = format_output(result3)
    budget_summary3 = calculate_budget_summary(formatted3, example3_input["budget_per_person"])
    display_results(formatted3, budget_summary3, result3["destination"])
else:
    print(f"❌ Error: {result3.get('error', 'Unknown error')}")

## Section 10: Debugging & Advanced Features

### Debug Mode - View Raw Agent Responses

In [ ]:
# Run with debug mode to see raw responses
debug_input = {
    "activities": [
        "British Museum",
        "Tower of London"
    ],
    "city": "London",
    "num_days": 2,
    "budget_per_person": 150.0
}

print("🔍 Running in DEBUG mode...\n")
debug_result = await find_activities(debug_input, debug=True)

if debug_result["status"] == "success":
    print("\n✓ Agent completed successfully")
    formatted_debug = format_output(debug_result)
    print(f"\nFormatted output ({len(formatted_debug)} activities):")
    print(json.dumps(formatted_debug, indent=2))

### Budget Tracking Analysis

In [ ]:
def analyze_budget_distribution(formatted_activities: List[Dict], num_days: int):
    """
    Analyze how activities are distributed across budget and categories.
    """
    print("\n📊 BUDGET DISTRIBUTION ANALYSIS")
    print("="*80)
    
    # By category
    category_costs = {}
    for activity in formatted_activities:
        cat = activity["category"]
        if cat not in category_costs:
            category_costs[cat] = 0
        category_costs[cat] += activity["price"]
    
    print("\nCost by Category:")
    for cat, cost in sorted(category_costs.items(), key=lambda x: x[1], reverse=True):
        print(f"  {cat:20s}: ${cost:.2f}")
    
    # Activity distribution
    total_activities = len(formatted_activities)
    activities_per_day = total_activities / num_days
    
    print(f"\nActivity Distribution:")
    print(f"  Total activities: {total_activities}")
    print(f"  Number of days: {num_days}")
    print(f"  Activities per day: {activities_per_day:.1f}")
    
    # Free vs Paid
    free_activities = [a for a in formatted_activities if a["price"] == 0]
    paid_activities = [a for a in formatted_activities if a["price"] > 0]
    
    print(f"\nFree vs Paid Activities:")
    print(f"  Free: {len(free_activities)} ({len(free_activities)/total_activities*100:.0f}%)")
    print(f"  Paid: {len(paid_activities)} ({len(paid_activities)/total_activities*100:.0f}%)")
    
    if paid_activities:
        avg_paid = sum(a["price"] for a in paid_activities) / len(paid_activities)
        print(f"  Average cost of paid activities: ${avg_paid:.2f}")
    
    print("="*80)

# Analyze Example 1 (Paris)
if 'formatted1' in locals():
    analyze_budget_distribution(formatted1, example1_input["num_days"])

### Custom Query Builder

In [ ]:
def quick_search(city: str, activities: List[str], num_days: int = 3, budget: float = 300.0):
    """
    Quick helper function to search for activities.
    """
    input_data = {
        "activities": activities,
        "city": city,
        "num_days": num_days,
        "budget_per_person": budget
    }
    
    result = await find_activities(input_data, debug=False)
    
    if result["status"] == "success":
        formatted = format_output(result)
        budget_summary = calculate_budget_summary(formatted, budget)
        display_results(formatted, budget_summary, result["destination"])
        return formatted
    else:
        print(f"❌ Error: {result.get('error', 'Unknown error')}")
        return None

print("✓ quick_search() helper function defined")
print("\nUsage example:")
print("  results = await quick_search('Rome', ['Colosseum', 'Vatican Museums'], num_days=2, budget=200)")

### Validation & Testing

In [ ]:
def validate_results(result: Dict, input_data: Dict):
    """
    Validate that agent results meet requirements.
    """
    print("\n🔍 VALIDATION RESULTS")
    print("="*80)
    
    checks = []
    
    # Check 1: All input activities have corresponding outputs
    if result["status"] == "success":
        input_activities = set(input_data["activities"])
        output_activities = set(a["name"] for a in result["activities"])
        
        all_present = input_activities.issubset(output_activities)
        checks.append(("All activities present", all_present))
        
        # Check 2: All activities have valid prices (>= 0)
        valid_prices = all(a["estimated_cost_per_person"] >= 0 for a in result["activities"])
        checks.append(("Valid prices (>= 0)", valid_prices))
        
        # Check 3: All activities have durations
        has_durations = all(a["estimated_duration"] for a in result["activities"])
        checks.append(("All have durations", has_durations))
        
        # Check 4: All activities have categories
        has_categories = all(a["category"] for a in result["activities"])
        checks.append(("All have categories", has_categories))
        
        # Check 5: Budget check
        total_cost = sum(a["estimated_cost_per_person"] for a in result["activities"])
        within_budget = total_cost <= input_data["budget_per_person"] * 1.1  # 10% tolerance
        checks.append(("Within budget (±10%)", within_budget))
    else:
        checks.append(("Agent success", False))
    
    # Display results
    for check_name, passed in checks:
        status = "✓" if passed else "✗"
        print(f"  {status} {check_name}")
    
    all_passed = all(passed for _, passed in checks)
    print("\n" + "="*80)
    if all_passed:
        print("✓ All validation checks passed!")
    else:
        print("⚠️  Some validation checks failed")
    print("="*80)
    
    return all_passed

# Validate Example 1 if it exists
if 'result1' in locals():
    validate_results(result1, example1_input)

## Summary

This notebook provides a complete implementation of the Activity Finder agent using Google ADK with dual search capabilities (Google Search + Tavily Search).

### Key Features:
- ✓ Google Search integration for primary activity research
- ✓ Tavily Search integration for enrichment and verification
- ✓ Budget tracking and validation
- ✓ Structured output format (title, price, description, duration, category)
- ✓ Multiple example test cases
- ✓ Debug mode for troubleshooting
- ✓ Budget distribution analysis
- ✓ Validation checks

### Usage:
1. Ensure API keys are set (GOOGLE_API_KEY, TAVILY_API_KEY)
2. Run all cells in order
3. Use provided examples or `quick_search()` for custom queries
4. Enable debug mode to see raw agent responses

### Output Format:
```python
[
  {
    "title": "Activity Name",
    "price": 35.00,
    "description": "Detailed description...",
    "duration": "2-3 hours",
    "category": "sightseeing"
  },
  ...
]
```